<div align="center" style="border:solid 1px gray;">
    <a href="https://openalex.org/">
        <img src="../../resources/img/OpenAlex-banner.png" alt="OpenAlex banner" width="300">
    </a>
</div>

# Getting started with OpenAlex Premium

In this tutorial, we're going to learn how to get started using [OpenAlex Premium](https://openalex.org/pricing). This subscription service provides some features beyond the free services. One of the most important of these features is **faster updates,** allowing you to keep your data fully synced with OpenAlex.

The way we do this is by using the `from_created_date` [(doc)](https://docs.openalex.org/api-entities/works/filter-works#from_created_date) or the `from_updated_date` [(doc)](https://docs.openalex.org/api-entities/works/filter-works#from_updated_date) filters. These filters allow you to get the new works you need to keep your data updated, and they require a Premium API Key to work.

We're going to set up the code to poll the OpenAlex API for newly updated works on a regular basis, once per day.

First, we need to get the API key you received by signing up for OpenAlex Premium. (Don't have a key yet? [Contact us right now to learn more about getting premium!](https://openalex.org/pricing))

We'll store our API key in a variable called `my_api_key`. There are several ways to do this. You could just put it into the code, but since it is sensitive information that we don't want others to see, we're going to get it from an [environment variable, which we'll store in a `.env` file.](https://towardsdatascience.com/the-quick-guide-to-using-environment-variables-in-python-d4ec9291619e)

This is just a text file with the name `.env`, that looks like this:
```
API_KEY=<my-api-key>
```
Replace `<my-api-key>` with your OpenAlex Premium API Key.


Now, to set our `my_api_key` variable, we'll set our environment using the [`python-dotenv`](https://pypi.org/project/python-dotenv/) library, then get the variable using the `os.getenv()` function.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv('.env')
my_api_key = os.getenv('API_KEY')
if my_api_key is None:
    print("No API key found!!!")
else:
    print("API key is set!")

API key is set!


Our plan is to get all of the works that have been updated in the last four hours. So let's construct a URL that will request that information from the API:

In [4]:
import requests
from datetime import datetime, timedelta

To use the API key, we have two options. The first option is to include it in the URL, as an `api_key` parameter:

In [9]:
four_hours_ago = datetime.utcnow() - timedelta(hours=4)
four_hours_ago_formatted_string = four_hours_ago.isoformat()
print(f"Our formatted date-time string looks like this: {four_hours_ago_formatted_string}")
# Construct a URL to requests works from the last four hours, including our API key as a URL query parameter.
url = f"https://api.openalex.org/works?filter=from_updated_date:{four_hours_ago_formatted_string}&api_key={my_api_key}"
print(f"Requesting newly updated works, including the API key as a URL query parameter...")
r = requests.get(url)
updated_works = r.json()

count_works_retrieved = len(updated_works['results'])
count_works_total = updated_works['meta']['count']
print(f"Retrieved {count_works_retrieved} works, out of {count_works_total} works updated since {four_hours_ago_formatted_string}")


Our formatted date-time string looks like this: 2023-11-07T19:22:46.848349
Requesting newly updated works, including the API key as a URL query parameter...


Retrieved 25 works, out of 1967482 works updated since 2023-11-07T19:22:46.848349


Success! We've used our API key to request all of the works updated in the last day. To get all of the data, [you can page through the results](https://docs.openalex.org/how-to-use-the-api/get-lists-of-entities/paging).

Alternatively, we can include the API key as a request header:

In [10]:
# Requests works from the last four hours, including our API key in the request headers
url = f"https://api.openalex.org/works?filter=from_updated_date:{four_hours_ago_formatted_string}&per-page=200"
headers = {"api_key": my_api_key}
print(f"Requesting newly updated works, including the API key in the request headers...")
r = requests.get(url, headers=headers)
updated_works = r.json()

count_works_retrieved = len(updated_works['results'])
count_works_total = updated_works['meta']['count']
print(f"Retrieved {count_works_retrieved} works, out of {count_works_total} works updated since {four_hours_ago_formatted_string}")


Requesting newly updated works, including the API key in the request headers...
Retrieved 200 works, out of 1965510 works updated since 2023-11-07T19:22:46.848349


You can retrieve up to 200 works per page. Again, to get all of the results, [you can page through the results](https://docs.openalex.org/how-to-use-the-api/get-lists-of-entities/paging).

Keep in mind that this method will get works that have been updated with *any change at all*, including increases in various counts. If you're only interested in *new* works, you could use the [`from_created_date`](https://docs.openalex.org/api-entities/works/filter-works#from_created_date) filter instead of `from_updated_date`, which will give a much smaller number of works.

You'll need to do two things to keep your data fresh:
1. Set up a script that does something similar to what we did above, and that runs on schedule once per day (using a [cron job](https://en.wikipedia.org/wiki/Cron), for example).
2. Update your database with the new data you've grabbed from our API (such as using a SQL script).

And that's it! Using this method, you can keep your data up to date with regular API requests, instead of waiting for new data snapshots.

Enjoy!